<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/hans-on-ml/rnn-01-univariate-time-series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lot


In [34]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from io import BytesIO
import requests

In [35]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0,1,n_steps)
    series = 0.5 * np.sin((time-offset1) * (freq1*10+10)) # 사인곡선 1
    series += 0.2 * np.sin((time-offset2) * (freq2*20+20)) # + 사인곡선 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + 잡음
    return series[..., np.newaxis].astype(np.float32)

In [36]:
n_step = 50
series = generate_time_series(10000, n_step+1)
x_train, y_train = series[:7000, :n_step], series[:7000, -1]
x_valid, y_valid = series[7000:9000, :n_step], series[7000:9000, -1]
x_test, y_test = series[9000:, :n_step], series[9000:, -1]

In [37]:
x_train.shape

(7000, 50, 1)

In [38]:
# 기준성능
# naive forecasting
y_pred = x_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020139335

In [44]:
# 기준성능
# Full-connected 선형모델
# 1차원 특성 배열을 기대하기 때문에 Flatten 층 추가
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50,1]),
    keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid), verbose=0)
loss = np.mean(history.history['val_loss'])
print(f'loss: {loss}')

loss: 0.00904791122302413


In [45]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_12 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 51        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [42]:
model = keras.models.Sequential([
  keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid), verbose=0)
loss = np.mean(history.history['val_loss'])
print(f'loss: {loss}')

loss: 0.010827633272856473


In [43]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [47]:
# 심층 RNN1
model = keras.models.Sequential([
  keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
  keras.layers.SimpleRNN(20, return_sequences=True),
  keras.layers.SimpleRNN(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid), verbose=0)
loss = np.mean(history.history['val_loss'])
print(f'loss: {loss}')

loss: 0.0054742124862968925


In [48]:
# # 심층 RNN2
model = keras.models.Sequential([
  keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
  keras.layers.SimpleRNN(20),
  keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid), verbose=0)
loss = np.mean(history.history['val_loss'])
print(f'loss: {loss}')

loss: 0.003186594194266945
